<a href="https://colab.research.google.com/github/saiteja15-cloud/DATA-SCIENCE/blob/main/NLP_and_Language_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Stochastic HW4/'''

"from google.colab import drive\ndrive.mount('/content/drive')\n%cd /content/drive/My Drive/Stochastic HW4/"

In [ ]:
%pylab inline
import math
import numpy.linalg as LA
import codecs, json
from time import time
import plotly.express as px
import re
import string
import requests
import collections
from collections import Counter

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['time']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +




---



---


# Q3: Natural Language Processing (****) (60pt)


One of applications of Markov chain in natural language processing is language model. In particular, we will work on uni-gram and bi-gram.

The following data set provides you with the training data files (a subset of the One Billion Word Language Modeling Bench-
mark). Each line in each file contains a whitespace-tokenized sentence.

- `1b benchmark.train.tokens`: data for training your language models.
- `1b benchmark.dev.tokens`: data for debugging and choosing the best hyperparameters.
- `1b benchmark.test.tokens`: data for evaluating your language models.


I have precessed these dataset for the purposed of this problem. The total number of unique words is 80661. In bi-gram model, the size of the transition matrix is $80661\times 80661$. It is impossible the store these matrices directly and we have to take advantage of the sparsity of the transition matrix.

You should use the development data to choose the best values for the hyperparameters $\lambda$. Hyperparameter
optimization is an active area of research; for this homework, you can simply try a few combinations to find
reasonable values.

In [ ]:
!wget https://raw.githubusercontent.com/yexf308/AppliedStochasticProcess/main/HW/HW2/1b_benchmark.train.tokens?raw=true -O 1b_benchmark.train.tokens
!wget https://raw.githubusercontent.com/yexf308/AppliedStochasticProcess/main/HW/HW2/1b_benchmark.dev.tokens?raw=true -O 1b_benchmark.dev.tokens
!wget https://raw.githubusercontent.com/yexf308/AppliedStochasticProcess/main/HW/HW2/1b_benchmark.test.tokens?raw=true -O 1b_benchmark.test.tokens


--2023-11-11 13:36:06--  https://raw.githubusercontent.com/yexf308/AppliedStochasticProcess/main/HW/HW2/1b_benchmark.train.tokens?raw=true
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8422714 (8.0M) [text/plain]
Saving to: ‘1b_benchmark.train.tokens’

1b_benchmark.train. 100%[===================>]   8.03M  --.-KB/s    in 0.1s    

2023-11-11 13:36:06 (75.2 MB/s) - ‘1b_benchmark.train.tokens’ saved [8422714/8422714]

--2023-11-11 13:36:06--  https://raw.githubusercontent.com/yexf308/AppliedStochasticProcess/main/HW/HW2/1b_benchmark.dev.tokens?raw=true
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:4

In [ ]:
START_TOKEN = "<s>"
STOP_TOKEN = "</s>"

class FileParser:
    def __init__(self, train_file="1b_benchmark.train.tokens", test_file="1b_benchmark.test.tokens", dev_file="1b_benchmark.dev.tokens"):
        self.TRAIN_FILE = train_file
        self.TEST_FILE  = test_file
        self.DEV_FILE = dev_file

    def get_train_file_tokens(self):
        return self._tokenize(self._get_sentences(self.TRAIN_FILE))

    def get_dev_file_tokens(self):
        return self._tokenize(self._get_sentences(self.DEV_FILE))

    def get_test_file_tokens(self):
        return self._tokenize(self._get_sentences(self.TEST_FILE))

    def get_train_file_sentence_tokens(self):
        return self._tokenize(self._get_sentences(self.TRAIN_FILE), flatten=False)

    def get_dev_file_sentence_tokens(self):
        return self._tokenize(self._get_sentences(self.DEV_FILE), flatten=False)

    def get_test_file_sentence_tokens(self):
        return self._tokenize(self._get_sentences(self.TEST_FILE), flatten=False)

    def _flatten(self, l):
        return [word for sublist in l for word in sublist]

    def _tokenize(self, sentence_list, flatten=True):
        tokenized_sentences = [re.split("\s+", sentence.strip()) for sentence in sentence_list]

        if flatten:
            return self._flatten(tokenized_sentences)

        return tokenized_sentences

    def _get_sentences(self, file_path):

        l = []
        with open(file_path, "r") as f:
            l = f.readlines()

        # Add the start and stop tokens to each sentence in the file
        sentence_list = []
        for sentence in l:
            sentence_list.append(START_TOKEN + " " + sentence + " " + STOP_TOKEN)

        return sentence_list

In [ ]:
fp = FileParser()
train_tokens = fp.get_train_file_sentence_tokens()
dev_tokens = fp.get_dev_file_sentence_tokens()
test_tokens = fp.get_test_file_sentence_tokens()
# The fourth line of training set
print(train_tokens[3])

['<s>', 'Abu', 'Dhabi', 'is', 'going', 'ahead', 'to', 'build', 'solar', 'city', 'and', 'no', 'pollution', 'city', '.', '</s>']


In [ ]:
# Building the dict and the map from the words to integers for the purpose of training.
word2idx={}
idx2word={}
idx=0
for line in train_tokens:
    for word in line:
        if word not in word2idx:
            word2idx[word]=idx
            idx2word[idx]=word
            idx+=1

# Transform the words in each sentence to integers
train_data=[]
for line in train_tokens:
    train_data.append([word2idx[word] for word in line])

# there are 80661 unique words.
len(word2idx)

80661



---



---


**Your task:** Complete the code in the ```LanguageModel``` class as follows.

1. (15pt) Implement **Laplace Smoothing** with the smoothing factor =1 in the function ```get_unigram_probability``` and ```get_bigram_probability```. In the bigram model, try to implement it as efficient as possible. **Hint:** For bigrams $(t_1,t_2)$ which do not occur in the sample, what is $p(t_2|t_1)$? For fixed $t_1$, are these probability the same?

2. (15pt) Complete functions ```get_unigram_sentence_log_probability``` and ```get_bigram_sentence_log_probability``` to calculate the log probability of the given sentence. You need to consider the situation that when Laplace Smoothing is true and the situation that when Laplace Smoothing is false.


3. (10pt) To make your language model work better, you will implement linear interpolation smoothing between
unigram, bigram.
\begin{align}
p'(t_2|t_1) = \lambda_1 p(t_2) + \lambda_2 p(t_2|t_1)
\end{align}
where $p'$ represents the smoothed probability, the hyperparameters $\lambda_1, \lambda_2$ are weights on the unigram,
bigram language models, respectively. So $\lambda_1+\lambda_2= 1$.
Complete functions ```get_linear_interpolation_probability``` and ```get_linear_interpolation_sentence_log_probability``` with $\mm\lambda = (\lambda_1, \lambda_2)$ stored in ```linear_interpolation_factors```.

4. (5pt) Testing the smoothed probability with several sentences in the development dataset. You should test when laplace smoothing is True and when laplace smoothing is False. You might find some words in testing dataset are not appeared in training set and you can set a default probability for this situation.  

3. (15pt) Report the **perplexity scores** of the linear interpolation of language model for your training,
and development sets. Report no
more than 5 different sets of $\lambda$. Briefly discuss the experimental results. Putting it all together, report perplexity on the test set, using the hyperparameters that you chose from
the development set. Specify those hyperparameters.

In [ ]:
class LanguageModel:
    def __init__(self, laplace_smoothing=False, laplace_smoothing_factor=1, linear_interpolation_factors=(0.3, 0.7)):
        self.unigram_freqs = {}
        self.unigram_corpus_length = 0
        self.num_unique_unigrams = 0

        self.bigram_freqs = {}
        self.bigram_corpus_length = 0
        self.num_unique_bigrams = 0

        self.laplace_smoothing = laplace_smoothing
        self.laplace_smoothing_factor = laplace_smoothing_factor
        self.linear_interpolation_factors = linear_interpolation_factors

    def fit_unigram(self, tokens):
        for clist in tokens:
            for cindex in range(len(clist)):
                t = clist[cindex]
                self.unigram_freqs[t] = self.unigram_freqs.get(t, 0) + 1

        self.unigram_corpus_length = sum(list(self.unigram_freqs.values()))
        self.num_unique_unigrams = len(self.unigram_freqs)

    def fit_bigram(self, tokens):
        for clist in tokens:
            for cindex in range(len(clist) - 1):
                t1, t2 = clist[cindex], clist[cindex + 1]
                self.bigram_freqs[(t1, t2)] = self.bigram_freqs.get((t1, t2), 0) + 1

        self.bigram_corpus_length = sum(list(self.bigram_freqs.values()))
        self.num_unique_bigrams = len(self.bigram_freqs)

    def get_unigram_probability(self, unigram):

        # With Laplace Smoothing -
        if self.laplace_smoothing:
            prob_numerator = self.unigram_freqs.get(unigram, 0) + self.laplace_smoothing_factor
            prob_denominator = self.unigram_corpus_length + (self.laplace_smoothing_factor * self.num_unique_unigrams)

        # Without Laplace Smooothing -
        else:
            prob_numerator = self.unigram_freqs.get(unigram, 0)
            prob_denominator = self.unigram_corpus_length

        prob = float(prob_numerator) / float(prob_denominator)
        return prob

    def get_bigram_probability(self, bigram):
        t1, t2 = bigram
        default_prob = 0.001
        # With Laplace Smooothing -
        if self.laplace_smoothing:
            prob_numerator = self.bigram_freqs.get((t1, t2), 0) + self.laplace_smoothing_factor
            prob_denominator = self.unigram_freqs.get(t1, 0) + (self.laplace_smoothing_factor * self.num_unique_unigrams)

        # Without Laplace Smooothing -
        else:
            prob_numerator = self.bigram_freqs.get((t1, t2), 0)
            prob_denominator = self.unigram_freqs.get(t1, 0)

        if prob_denominator == 0:
          #print(f"Error: 'get_bigram_probability()' has a denominator of 0 for {bigram}")
          return default_prob

        prob = float(prob_numerator) / float(prob_denominator)
        return prob

    def get_linear_interpolation_probability(self, bigram):
        lambda_1, lambda_2 = self.linear_interpolation_factors
        unigram_prob = self.get_unigram_probability(bigram[1])  # p(t2) SMOOTHING = TRUE
        bigram_prob = self.get_bigram_probability(bigram)
        return (lambda_1 * unigram_prob) + (lambda_2 * bigram_prob)

    def get_unigram_sentence_log_probability(self, sentence):
        log_prob = 0.0
        for word in sentence:
            log_prob += math.log(self.get_unigram_probability(word)) # smoothing=self.laplace_smoothing
        return log_prob

    def get_bigram_sentence_log_probability(self, sentence):
        log_prob = 0.0
        for i in range(len(sentence) - 1):
            bigram = (sentence[i], sentence[i + 1])
            log_prob += math.log(self.get_bigram_probability(bigram)) # smoothing=self.laplace_smoothing
        return log_prob

    def get_linear_interpolation_sentence_log_probability(self, sentence):
        log_prob = 0.0
        for i in range(len(sentence) - 1):
            bigram = (sentence[i], sentence[i + 1])
            log_prob += math.log(self.get_linear_interpolation_probability(bigram))
        return log_prob

    def get_unigram_perplexity(self, sentences):
        total_log_prob = 0.0
        total_words = 0
        for sentence in sentences:
            total_words += len(sentence)
            total_log_prob += self.get_unigram_sentence_log_probability(sentence)
        return math.exp(-total_log_prob / total_words)

    def get_bigram_perplexity(self, sentences):
        total_log_prob = 0.0
        total_words = 0
        for sentence in sentences:
            total_words += len(sentence)
            total_log_prob += self.get_bigram_sentence_log_probability(sentence)
        return math.exp(-total_log_prob / total_words)

    def get_linear_interpolation_perplexity(self, sentences):
        total_log_prob = 0.0
        total_words = 0
        for sentence in sentences:
            total_words += len(sentence)
            total_log_prob += self.get_linear_interpolation_sentence_log_probability(sentence)
        return math.exp(-total_log_prob / total_words)

# Ref - Class notes, Course Github codes

In [ ]:
# 3.4

lm_with_smoothing = LanguageModel(laplace_smoothing=True, laplace_smoothing_factor=1)
lm_without_smoothing = LanguageModel()

lm_with_smoothing.fit_unigram(train_data)
lm_with_smoothing.fit_bigram(train_data)

lm_without_smoothing.fit_unigram(train_data)
lm_without_smoothing.fit_bigram(train_data)

print("Unigram Probabilities without smoothing-")
for sentence in dev_tokens[1:2]:
    for word in sentence:
        prob_without_smoothing = lm_without_smoothing.get_unigram_probability(word)
        print(f"Word: {word}, Probability without Smoothing: {prob_without_smoothing}")

print("\nUnigram probabilities with smoothing-")
for sentence in dev_tokens[1:2]:
    for word in sentence:
        prob_with_smoothing = lm_with_smoothing.get_unigram_probability(word)
        print(f"Word: {word}, Probability with Smoothing: {prob_with_smoothing}")

Unigram Probabilities without smoothing-
Word: <s>, Probability without Smoothing: 0.0
Word: Everybody, Probability without Smoothing: 0.0
Word: is, Probability without Smoothing: 0.0
Word: aware, Probability without Smoothing: 0.0
Word: of, Probability without Smoothing: 0.0
Word: consequences, Probability without Smoothing: 0.0
Word: ., Probability without Smoothing: 0.0
Word: </s>, Probability without Smoothing: 0.0

Unigram probabilities with smoothing-
Word: <s>, Probability with Smoothing: 5.665414232427018e-07
Word: Everybody, Probability with Smoothing: 5.665414232427018e-07
Word: is, Probability with Smoothing: 5.665414232427018e-07
Word: aware, Probability with Smoothing: 5.665414232427018e-07
Word: of, Probability with Smoothing: 5.665414232427018e-07
Word: consequences, Probability with Smoothing: 5.665414232427018e-07
Word: ., Probability with Smoothing: 5.665414232427018e-07
Word: </s>, Probability with Smoothing: 5.665414232427018e-07


In [ ]:
print("Bigram probabilities with smoothing-")
for sentence in dev_tokens[1:2]:
    for i in range(len(sentence) - 1):
        bigram = (sentence[i], sentence[i + 1])
        prob_without_smoothing = lm_without_smoothing.get_bigram_probability(bigram)
        print(f"Bigram: {bigram}, Probability without Smoothing: {prob_without_smoothing}")

print("\nBigram probabilities with smoothing-")
for sentence in dev_tokens[1:2]:
    for i in range(len(sentence) - 1):
        bigram = (sentence[i], sentence[i + 1])
        prob_with_smoothing = lm_with_smoothing.get_bigram_probability(bigram)
        print(f"Bigram: {bigram}, Probability without Smoothing: {prob_with_smoothing}")

Bigram probabilities with smoothing-
Bigram: ('<s>', 'Everybody'), Probability without Smoothing: 0.001
Bigram: ('Everybody', 'is'), Probability without Smoothing: 0.001
Bigram: ('is', 'aware'), Probability without Smoothing: 0.001
Bigram: ('aware', 'of'), Probability without Smoothing: 0.001
Bigram: ('of', 'consequences'), Probability without Smoothing: 0.001
Bigram: ('consequences', '.'), Probability without Smoothing: 0.001
Bigram: ('.', '</s>'), Probability without Smoothing: 0.001

Bigram probabilities with smoothing-
Bigram: ('<s>', 'Everybody'), Probability without Smoothing: 1.2397565118210784e-05
Bigram: ('Everybody', 'is'), Probability without Smoothing: 1.2397565118210784e-05
Bigram: ('is', 'aware'), Probability without Smoothing: 1.2397565118210784e-05
Bigram: ('aware', 'of'), Probability without Smoothing: 1.2397565118210784e-05
Bigram: ('of', 'consequences'), Probability without Smoothing: 1.2397565118210784e-05
Bigram: ('consequences', '.'), Probability without Smoothing

In [ ]:
# 3.5

lm_linear_interpolation = LanguageModel(laplace_smoothing=True, laplace_smoothing_factor=1)
lm_linear_interpolation.fit_unigram(train_data)
lm_linear_interpolation.fit_bigram(train_data)

# Define different sets of lambda values
lambda_sets = [(0.2, 0.8), (0.4, 0.6), (0.5, 0.5), (0.6, 0.4), (0.8, 0.2)]

# Compute perplexity scores for each set of lambda values on the training and development sets
for lambdas in lambda_sets:
    lm_linear_interpolation.linear_interpolation_factors = lambdas

    train_perplexity = lm_linear_interpolation.get_linear_interpolation_perplexity(train_data)
    dev_perplexity = lm_linear_interpolation.get_linear_interpolation_perplexity(dev_tokens)

    print(f"Lambda values: {lambdas}")
    print(f"Training Perplexity: {train_perplexity}")
    print(f"Development Perplexity: {dev_perplexity}")
    print("\n")


Lambda values: (0.2, 0.8)
Training Perplexity: 1366.0797982191657
Development Perplexity: 65511.71145427921


Lambda values: (0.4, 0.6)
Training Perplexity: 1132.2554569040485
Development Perplexity: 84897.75398931614


Lambda values: (0.5, 0.5)
Training Perplexity: 1078.6091091879275
Development Perplexity: 99781.25404446606


Lambda values: (0.6, 0.4)
Training Perplexity: 1048.2493245779701
Development Perplexity: 121165.60100657163


Lambda values: (0.8, 0.2)
Training Perplexity: 1048.369270521507
Development Perplexity: 214251.55663371072




In [ ]:
# Choose the lowest development perplexity score
best_lambda_values = (0.2, 0.8)
lm_linear_interpolation.linear_interpolation_factors = best_lambda_values

# Compute perplexity on the test set
test_perplexity = lm_linear_interpolation.get_linear_interpolation_perplexity(test_tokens)
print(f"Best Lambda Values: {best_lambda_values}")
print(f"Test Perplexity: {test_perplexity}")


Best Lambda Values: (0.2, 0.8)
Test Perplexity: 65387.9848265971


# Q4: (Bonus) Beam Search Revisited (*****) (15pt)
Implement Beam Search Algorithm for Q3. Note you need to use various data structure and graphs to reduce the complexity of implementation.


In [ ]:
import math
import heapq

class LanguageModel:
    def __init__(self, laplace_smoothing=False, laplace_smoothing_factor=1, linear_interpolation_factors=(0.3, 0.7)):
        self.unigram_freqs = {}
        self.unigram_corpus_length = 0
        self.num_unique_unigrams = 0

        self.bigram_freqs = {}
        self.bigram_corpus_length = 0
        self.num_unique_bigrams = 0

        self.laplace_smoothing = laplace_smoothing
        self.laplace_smoothing_factor = laplace_smoothing_factor
        self.linear_interpolation_factors = linear_interpolation_factors

    def fit_unigram(self, tokens):
        for clist in tokens:
            for cindex in range(len(clist)):
                t = clist[cindex]
                self.unigram_freqs[t] = self.unigram_freqs.get(t, 0) + 1

        self.unigram_corpus_length = sum(list(self.unigram_freqs.values()))
        self.num_unique_unigrams = len(self.unigram_freqs)

    def fit_bigram(self, tokens):
        for clist in tokens:
            for cindex in range(len(clist) - 1):
                t1, t2 = clist[cindex], clist[cindex + 1]
                self.bigram_freqs[(t1, t2)] = self.bigram_freqs.get((t1, t2), 0) + 1

        self.bigram_corpus_length = sum(list(self.bigram_freqs.values()))
        self.num_unique_bigrams = len(self.bigram_freqs)

    def get_unigram_probability(self, unigram):

        # With Laplace Smoothing -
        if self.laplace_smoothing:
            prob_numerator = self.unigram_freqs.get(unigram, 0) + self.laplace_smoothing_factor
            prob_denominator = self.unigram_corpus_length + (self.laplace_smoothing_factor * self.num_unique_unigrams)

        # Without Laplace Smooothing -
        else:
            prob_numerator = self.unigram_freqs.get(unigram, 0)
            prob_denominator = self.unigram_corpus_length

        prob = float(prob_numerator) / float(prob_denominator)
        return prob

    def get_bigram_probability(self, bigram):
        t1, t2 = bigram

        # With Laplace Smooothing -
        if self.laplace_smoothing:
            prob_numerator = self.bigram_freqs.get((t1, t2), 0) + self.laplace_smoothing_factor
            prob_denominator = self.unigram_freqs.get(t1, 0) + (self.laplace_smoothing_factor * self.num_unique_unigrams)

        # Without Laplace Smooothing -
        else:
            prob_numerator = self.bigram_freqs.get((t1, t2), 0)
            prob_denominator = self.unigram_freqs.get(t1, 0)

        if prob_denominator == 0:
          print(f"Error: 'get_bigram_probability()' has a denominator of 0 for {bigram}")
          return 0

        prob = float(prob_numerator) / float(prob_denominator)
        return prob

    def get_linear_interpolation_probability(self, bigram):
        lambda_1, lambda_2 = self.linear_interpolation_factors
        unigram_prob = self.get_unigram_probability(bigram[1])  # p(t2) SMOOTHING = TRUE
        bigram_prob = self.get_bigram_probability(bigram)
        return (lambda_1 * unigram_prob) + (lambda_2 * bigram_prob)


    def get_unigram_sentence_log_probability(self, sentence):
        log_prob = 0.0
        for word in sentence:
            log_prob += math.log(self.get_unigram_probability(word)) # smoothing=self.laplace_smoothing
        return log_prob

    def get_bigram_sentence_log_probability(self, sentence):
        log_prob = 0.0
        for i in range(len(sentence) - 1):
            bigram = (sentence[i], sentence[i + 1])
            log_prob += math.log(self.get_bigram_probability(bigram)) # smoothing=self.laplace_smoothing
        return log_prob

    def get_linear_interpolation_sentence_log_probability(self, sentence):
        log_prob = 0.0
        for i in range(len(sentence) - 1):
            bigram = (sentence[i], sentence[i + 1])
            log_prob += math.log(self.get_linear_interpolation_probability(bigram))
        return log_prob

    def get_unigram_perplexity(self, sentences):
        total_log_prob = 0.0
        total_words = 0
        for sentence in sentences:
            total_words += len(sentence)
            total_log_prob += self.get_unigram_sentence_log_probability(sentence)
        return math.exp(-total_log_prob / total_words)

    def get_bigram_perplexity(self, sentences):
        total_log_prob = 0.0
        total_words = 0
        for sentence in sentences:
            total_words += len(sentence)
            total_log_prob += self.get_bigram_sentence_log_probability(sentence)
        return math.exp(-total_log_prob / total_words)

    def get_linear_interpolation_perplexity(self, sentences):
        total_log_prob = 0.0
        total_words = 0
        for sentence in sentences:
            total_words += len(sentence)
            total_log_prob += self.get_linear_interpolation_sentence_log_probability(sentence)
        return math.exp(-total_log_prob / total_words)

    def beam_search(self, start_token='<s>', stop_token='</s>', beam_width=5, max_length=20):
        beam = [([], 0.0)]  # (hypothesis, log probability)

        for _ in range(max_length):
            candidates = []

            for hypothesis, log_prob in beam:
                current_word = hypothesis[-1] if hypothesis else start_token

                if current_word == stop_token:
                    candidates.append((hypothesis, log_prob))
                    continue

                next_words = self.get_next_words(current_word, beam_width)
                for next_word in next_words:
                    new_hypothesis = hypothesis + [next_word]
                    new_log_prob = log_prob + math.log(self.get_linear_interpolation_probability((current_word, next_word)))

                    candidates.append((new_hypothesis, new_log_prob))

            beam = heapq.nlargest(beam_width, candidates, key=lambda x: x[1])

        return beam

    def get_next_words(self, current_word, beam_width):
        word_probabilities = [(word, self.get_linear_interpolation_probability((current_word, word))) for word in self.unigram_freqs.keys()]
        sorted_words = sorted(word_probabilities, key=lambda x: x[1], reverse=True)
        next_words = [word for word, _ in sorted_words[:beam_width]]
        return next_words

In [ ]:
# Ref - Class notes, Course Github codes, Algorithms and Data Structures
lm = LanguageModel()
lm.fit_unigram(train_tokens)
lm.fit_bigram(train_tokens)

beam_width = 5
max_length = 10
lm.beam_search(start_token='fly', stop_token='</s>', beam_width=beam_width, max_length=max_length)

[(['.', '</s>'], -2.9190886823857496),
 (['to', 'the', ',', '"', '</s>'], -15.244650470702698),
 (['to', 'the', ',', '"', 'said', '.', '</s>'], -17.911989531129805),
 (['to', 'the', ',', '"', 'he', 'said', '.', '</s>'], -20.229702211080788),
 (['to', 'the', ',', '"', 'he', 'said', '.', ',', 'the', 'the'],
  -31.065786721335)]